<a href="https://colab.research.google.com/github/ssimpore/IBM-R/blob/master/Interface2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CODE**

In [1]:
from ipywidgets import Checkbox,Layout,VBox,HBox,Box,ToggleButton,Dropdown,Select,SelectMultiple,Button,Output,interactive_output,Label,FileUpload
from IPython.display import display
import pandas as pd
from datetime import datetime

In [2]:
class Filtre():
  test_affichage=True
  vbox_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='groove',
                    #width='auto',
                    height='100%',
                    width='auto'
                    )
  
  dvbox_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='groove',
                    #width='auto',
                    height='auto',
                    width='25%'
                    )
  
  b_layout = Layout(
      margin= '50% 0px 0px 25%'
      ,height='auto')
  
  label_layout=Layout(width='auto',
                               height='auto',
                               margin='0px') 
  
  liste_layout=Layout(width='auto',
                               height='auto',
                               margin= '0px 0px 0px 15%')

  box_layout = Layout(display='flex',
                    flex_flow='row',
                    #align_items='stretch',
                    border='solid',
                    width='auto',
                    height='auto',)   

  
  sortie_layout = Layout(display='flex',
                    flex_flow='column',
                    #align_items='stretch',
                    border='solid',
                    width='auto',
                    height='auto',)                      
  donnee={
      'techno':(['Solaire','Eolienne','Stockage'],'technologie'),
      'etat':(['Terminé', 'En cours', 'Planifiee','Annulé'],'état des accès'),
      'acces':(['AQ','DC','PC','IN','AD','TN','AC','NO','AU'],'type d'' accès'),
      'valideur':(['NSIMBA', 'SEYE', 'SIMPORE', 'AUERSWALD', 'CHEMIN', 'NDIR', 'RIBEIRO', 
                   'SIMON', 'VELCIN', 'WITKOWSKI'],"valideurs d'accès"),
      'scada':(['CONEXT','DARWIN','ENERGYSOFT','INGECON','NETECO','SUNXPER','WINCC','XFLOW'],"scada utilisé"),
      }

  button=Button(description='CALCULER',
      disabled=False,
      button_style='success', # 'success', 'info', 'warning', 'danger' or ''
      tooltip='Click me',
      icon='check' # (FontAwesome names without the `fa-` prefix)
      )
  
  def __init__(self,nom):
    donnee=Filtre.donnee
    self.nom=nom.lower()
    self.liste=list(map(lambda x: str(x).upper(),donnee[self.nom][0]))
    self.checkbox=[Checkbox(value=False,description=elt,disabled=False,indent=False)for elt in self.liste]
    self.description=donnee[self.nom][1].upper()
    self.label=Checkbox(value=True,description=self.description,disabled=False, indent=False)
    self.dropdown= Dropdown(options=self.liste,layout={'width': 'max-content'})
    self.label_vbox=label=VBox([self.label],layout=Filtre.label_layout)
    self.check_vbox=VBox(self.checkbox,layout=Filtre.liste_layout)
    self.vbox=VBox([self.label_vbox,self.check_vbox],layout=Filtre.vbox_layout)
    button=VBox([Filtre.button],layout=Filtre.b_layout)
    dvbox=Box([self.label,self.dropdown])
    self.dvbox=VBox([dvbox,button],layout=Filtre.dvbox_layout)
    self.message=''

  def make_choise(self):
    dict_objet={}
    liste=[]
    self.used=self.label.value
    nom=self.nom
    if self.nom!='scada':
      if self.used:
            for i in range(len(self.liste)):
              if self.checkbox[i].value:
                liste.append(self.liste[i])

    if self.nom=='scada':
      if self.used:
          liste.append(self.dropdown.value)
    self._liste=list(map(lambda x: str(x).lower(),liste))
    return self._liste,self.used

  def filter_by(self,df):
    try:
      df = df[df[self.nom.upper()].apply(lambda x: str(x).lower()).isin(self._liste)]
      assert len(self._liste) > 0
    except AssertionError:
      self.message=f"({self.description.capitalize()} non selectionné)"
    self.afficher(df)
    return df

  def afficher(self,df,description='',message=''):
    x=' '*(18-len(self.description))
    y=' '*(4-len(str(len(df))))
    if Filtre.test_affichage:
      print(f"=> Selection ({self.description.upper()}){x}=> {len(df)}{y}accès {self.message}")

  @classmethod
  def afficher_out(self,df,description='',message=''):
    x=' '*(18-len(description))
    y=' '*(4-len(str(len(df))))
    if Filtre.test_affichage:
      print(f"=> Selection ({description.upper()}){x}=> {len(df)}{y}accès {message}")

In [3]:
def import_data_frame(link_acces, link_pv):
    data_acces = pd.read_excel(link_acces, index_col=1)
    data_pv = pd.read_excel(link_pv, index_col=1)

    for index, row in data_acces.iterrows():
        if index in data_pv.index:
          data_acces.loc[index, 'Code'] = data_pv.loc[index, 'Code']
          data_acces.loc[index, 'scada'] = data_pv.loc[index, 'scada']
          data_acces.loc[index, 'autre_scada'] = data_pv.loc[index, 'autre_scada']
          data_acces.loc[index, 'Antenne'] = data_pv.loc[index, 'Antenne']
        else:
          print(f"Attention ==> Nouveau parc non enrégistré !{index}")
          
    df = data_acces
    df['acces_asso'] = df['Commentaire début'].apply(lambda x: str(x)[:2].upper())
    df = df.drop(columns=['Numéro'])  # remove mumero from data
    df.reset_index(drop=False, inplace=True)
    Col = {'Parc': 'parc', 'Type Maintenance': 'maintenance', 'Technologie': 'techno', 'Ouvrage': 'ouvrage',
           'Etat': 'etat', 'Valideur': 'valideur', 'Entreprise': 'entreprise', 'CA': 'ca',
           "Date d'autorisation": 'date_auto', "Date début": 'date_debut', "Date de fin": 'date_fin',
           'Commentaire début': 'comment_debut', 'Commentaire fin': 'comment_fin',
           'Interventions associées': 'intervention', 'Arrêts associés': 'arret_asso', 'acces_asso': 'acces',
           'Code': 'code', 'scada': 'scada', 'autre_scada': 'autre_scada', 'Antenne': 'Antenne'}

    newCol = {k: v.upper() for k, v in Col.items()}
    df.rename(columns=newCol, inplace=True)
    df.set_index('CODE', inplace=True)
    return df

In [4]:
def select_worked_data(df, work_file_limit):
  debut,fin=work_file_limit[0],work_file_limit[1]
  df=df[df['DATE_DEBUT'].apply(lambda x: (x >= debut) & (x <= fin))]
  Filtre.afficher_out(df,description='Aurora',message='')
  return df

In [5]:
def filtering_by_choise(df, Objets):
  for objets in Objets:
    objets.make_choise()
    if objets.label.value:
      df=objets.filter_by(df)
  return df

In [6]:
def data_with_calculation(df,horaire):
    indicateur=()
    def to_str(s):
      s=s.total_seconds()
      days,remainder = divmod(s, 86400)
      hours, remainder = divmod(remainder, 3600)
      minutes, seconds = divmod(remainder, 60)
      if s<86400:
        x='{:02}:{:02}:{:02}'.format(int(hours), int(minutes), int(seconds))
      else:
        x='{:02} jour(s),{:02}:{:02}:{:02}'.format(int(days),int(hours), int(minutes), int(seconds))
      return x

    def cal_indicateur(df,react,arret):
      react=to_str(df[react].mean()),to_str(df[react].min()),to_str(df[react].max())
      arret=to_str(df[arret].mean()),to_str(df[arret].min()),to_str(df[arret].max())
      return react,arret
      
    debut_job,fin_job=str(horaire[0]),str(horaire[1])

    def corr_debut_fin(current_date):
        hour, date = str(current_date)[11:], str(current_date)[:10]
        new_date=current_date
        in_range= (hour >= debut_job) &  (hour <= fin_job)
        trop_tot=hour < debut_job
        trop_tard=hour > fin_job
        if in_range:
          new_date=current_date
        if trop_tot:
          new_date=date + ' ' + debut_job
        if trop_tard:
          new_date=date + ' ' + fin_job
        return new_date
    def corr_auto(auto,debut):
        debut,auto=str(debut),str(auto)
        planned=auto<debut
        if planned:
          new_auto=debut
        else:
          new_auto=auto
        return new_auto

    def cal_react(debut, auto):
        dformat = "%d/%m/%Y %H:%M"
        debut=corr_debut_fin(debut)
        auto = corr_auto(auto,debut)
        react=datetime.strptime(auto, dformat) - datetime.strptime(debut, dformat)
        return react

    def cal_arret(debut, fin):
        dformat = "%d/%m/%Y %H:%M"
        arret=datetime.strptime(fin, dformat) - datetime.strptime(debut, dformat)
        return arret

    if len(df)>0:
      df['REACT']=df.apply(lambda x:cal_react(x['DATE_DEBUT'],x['DATE_AUTO']),axis=1)
      df['ARRET']=df.apply(lambda x:cal_react(x['DATE_DEBUT'],x['DATE_FIN']),axis=1)
      indicateur=cal_indicateur(df,'REACT','ARRET')
      df['REACT']=df['REACT'].apply(lambda x:to_str(x))
      df['ARRET']=df['ARRET'].apply(lambda x:to_str(x))
    else:
      print('-'*75 )
      message="! Attention base de données vide, indicateurs non calculés avec ce filtre ! ".upper()
      print(message)
      print('-'*75 )
      indicateur=''
    return indicateur

# **CALCUL**

In [7]:
Techno=Filtre('techno')
Etat=Filtre('etat')
Acces=Filtre('acces')
Valideur=Filtre('valideur')
Scada=Filtre('scada')

Techno.checkbox[0].value=True
Etat.checkbox[0].value=True
Acces.checkbox[0].value=True
Valideur.checkbox[0].value=True
Valideur.checkbox[1].value=True
Scada.label.value=False
button=Filtre.button
output=Output(),Output()
Titre=Label('Résultats'.upper()),Label('Données'.upper())
out=VBox([Titre[0],output[0]],layout=Filtre.sortie_layout),VBox([Titre[1],output[1]],layout=Filtre.sortie_layout)

C0=Scada.dvbox
C1=VBox([Techno.vbox,Etat.vbox])
C2=VBox([Acces.vbox])
C3=VBox([Valideur.vbox])
C=HBox([C0,C1,C2,C3],layout=Filtre.box_layout)
SCREEN=VBox([C,out[0],out[1]])
display(SCREEN)

def calculer():
  link_acces = "/content/drive/My Drive/Data/PV/exportAcces_2020_10_02.xls"
  link_pv = "/content/drive/My Drive/Data/PV/data_pv_v1.xls"

  plage = ('02/10/2020 00:00', '02/10/2020 23:59')
  horaire=('08:00','18:00')

  Objets=[Techno,Etat,Acces,Valideur,Scada]

  data = import_data_frame(link_acces, link_pv)
  data = select_worked_data(data, plage)
  data = filtering_by_choise(data, Objets)
  Indicateur=data_with_calculation(data,horaire)
  data.to_excel('/content/final_data.xls')

  with output[1]:
    display(data.iloc[:,[2,4,5,18,8,9,10,15]].head())
  
  return Indicateur

def on_button_clicked(b):
  with output[0]:
    Indicateur=calculer()
    print('-'*20)
    print(f' REACTIVITE:\n Moyenne  = {Indicateur[0][0]} \n Minimale = {Indicateur[0][1]} \n Maximale = {Indicateur[0][2]}')
    print('')
    print(f' ARRER EQUI:\n Moyenne  = {Indicateur[1][0]} \n Minimale = {Indicateur[1][1]} \n Maximale = {Indicateur[1][2]}')
    print('-'*20)
button.on_click(on_button_clicked)
